In [2]:
import numpy as np
from scipy.signal import cheby2, filtfilt, firwin, butter, hilbert

def bz_filter(samples, **kwargs):
    # Default values
    passband = kwargs.get('passband', [])
    stopband = kwargs.get('stopband', [])
    order = kwargs.get('order', 4)
    ripple = kwargs.get('ripple', 20)
    nyquist = kwargs.get('nyquist', 625)
    filter_type = kwargs.get('filter', 'cheby2')
    FMAlegacy = kwargs.get('FMAlegacy', False)
    BUZCODE = kwargs.get('BUZCODE', False)
    fast = kwargs.get('fast', False)
    channels = kwargs.get('channels', [])
    intervals = kwargs.get('intervals', [-np.inf, np.inf])

    # Check for valid inputs
    if not passband and not stopband:
        raise ValueError("Missing passband or stopband.")

    if 'samplingRate' in samples:
        nyquist = 0.5 * samples['samplingRate']

    if filter_type == 'cheby2':
        if passband:
            if passband[0] == 0:
                b, a = cheby2(order, ripple, passband[1] / nyquist, btype='low')
            elif passband[1] == np.inf:
                b, a = cheby2(order, ripple, passband[0] / nyquist, btype='high')
            else:
                b, a = cheby2(order, ripple, [i / nyquist for i in passband], btype='band')
        else:
            b, a = cheby2(order, ripple, [i / nyquist for i in stopband], btype='stop')
    elif filter_type == 'fir1':
        # Order input to fir1 needs to be in samples, not cycles
        if passband:
            if passband[0] == 0:
                filt_order = round(order * 2 * nyquist / passband[1])
                b = firwin(filt_order + 1, passband[1] / nyquist, pass_zero='lowpass')
            elif passband[1] == np.inf:
                filt_order = round(order * 2 * nyquist / passband[0])
                b = firwin(filt_order + 1, passband[0] / nyquist, pass_zero='highpass')
            else:
                filt_order = round(order * 2 * nyquist / passband[0])
                b = firwin(filt_order + 1, [i / nyquist for i in passband])
        else:
            filt_order = round(order * 2 * nyquist / stopband[0])
            b = firwin(filt_order + 1, [i / nyquist for i in stopband], pass_zero='bandstop')
        a = 1  # For FIR filter, a is 1
    elif filter_type == 'butter':
        if passband:
            b, a = butter(order, [i / nyquist for i in passband], btype='band')
        else:
            b, a = butter(order, stopband[0] / nyquist, btype='stop')
    else:
        raise ValueError(f"Unknown filter type: {filter_type}")

    if BUZCODE:
        # Handle BUZCODE structure (assuming it's a dictionary)
        pass  # Add the relevant conversion here
    elif FMAlegacy:
        # Handle FMAlegacy structure (assuming it's a numpy array or similar)
        pass  # Add the relevant conversion here
    else:
        # Handle as simple time series
        filtered = filtfilt(b, a, samples)

    return filtered

# Usage example:
samples = np.array([1,2,3,4,5])  # Your sample data
result = bz_filter(samples, passband=[0.1, 0.5], filter='fir1')


C:\Users\bkimq\AppData\Local\Temp\ipykernel_14896\1408584878.py:22: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'samplingRate' in samples:


ValueError: The length of the input vector x must be greater than padlen, which is 150003.